# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt

In [77]:
import ollama

model = "hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF"

# Download a model named "gemma3"
ollama.pull(model)

# Use it to chat
response = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Hello — how are you?"}]
)
print(response.message.content)

# Or generate with a prompt:
gen = ollama.generate(model=model, prompt="Tell me a joke.")
print(gen["response"])

Hey! I'm doing great, thanks for asking. 😊 How about you? How's your day going so far?
Why don't scientists trust atoms?

Because they make up *everything*!


# generating a new story

In [78]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

In [79]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)
premise_opening = response.message.content
Markdown(response.message.content)

Premise: A reclusive lighthouse keeper discovers that the mysterious radio signals he's been intercepting for years are actually transmissions from his future self, warning him about an impending disaster that only he can prevent—if he can figure out how to time travel without becoming trapped in the process.

Opening paragraph: Salt clung to the walls of the lighthouse like a second skin. Elias wiped it away with the back of his hand for the hundredth time that evening, leaving a temporary streak through the grime before it re-appeared as if by magic. The beam outside cut through the fog, its rhythmic pulse matching the thudding in his temples as he leaned over the radio console. Static hissed and crackled like distant surf, punctuated occasionally by snippets of voices—his own voice, speaking words that chilled him to the bone each time they repeated: "The storm comes. And I'm already here." For eleven years, this had been his only company in the remote tower, a beacon for ships he never saw and a listener to messages from nowhere. Tonight felt different, though, as if the very air hummed with possibility—and dread.

In [80]:
import re
pattern = r'(?mis)^\s*(?P<label>Premise|Original paragraph|Opening paragraph)\s*:\s*(?P<body>.+?)(?=\n{2,}|\n\S.*?:|$)'

sections = {}
for m in re.finditer(pattern, premise_opening):
    label = m.group('label')
    body = m.group('body').strip()
    sections[label] = f"{label}: {body}"  # includes the label in the value

premise = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

print(premise)
print(paragraph)

Premise: A reclusive lighthouse keeper discovers that the mysterious radio signals he's been intercepting for years are actually transmissions from his future self, warning him about an impending disaster that only he can prevent—if he can figure out how to time travel without becoming trapped in the process.
Opening paragraph: Salt clung to the walls of the lighthouse like a second skin. Elias wiped it away with the back of his hand for the hundredth time that evening, leaving a temporary streak through the grime before it re-appeared as if by magic. The beam outside cut through the fog, its rhythmic pulse matching the thudding in his temples as he leaned over the radio console. Static hissed and crackled like distant surf, punctuated occasionally by snippets of voices—his own voice, speaking words that chilled him to the bone each time they repeated: "The storm comes. And I'm already here." For eleven years, this had been his only company in the remote tower, a beacon for ships he ne

In [81]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [82]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)
continued_paragraph = response.message.content
Markdown(response.message.content)

His fingers hovered over the frequency dial, trembling slightly despite himself. The salt on his skin seemed to burn now, almost as if it were trying to remind him of something important. Elias had been waiting for a pattern, some clue that would help him make sense of these transmissions, but tonight was different. Tonight, he heard more than just fragments.

"Elias," the voice crackled through the static, clearer than ever before. "Listen carefully. The storm approaches on the solstice. You know what must be done."

The room seemed to tilt around him. His heart hammered against his ribs like a prisoner trying to escape. The solstice was three days away—a date he had circled on his calendar months ago, not because of any particular significance but because it marked the anniversary of his father's death.

He reached for the notebook he kept beside the radio, flipping through pages of scribbled notes from previous transmissions. "The storm comes. And I'm already here." Those words appeared repeatedly throughout the years, always with slight variations in tone and urgency.

"There has to be more," he muttered, turning up the volume slightly as if that would somehow reveal a secret hidden within the static.

As if in answer, the transmission changed again. "The lighthouse... it's not just a beacon... it's a key."

Elias's breath caught in his throat. He had spent years maintaining this tower, believing he was merely keeping ships safe from the rocky coast. But now? Now everything seemed to shift and rearrange itself in his mind.

He stood up abruptly, knocking over a cup of cold tea that stained the notebook pages like blood. The radio continued to hiss and crackle, but the clear voice had disappeared for now.

"I need answers," he said aloud to the empty room, though he wasn't sure if he was talking to himself or to the mysterious version of him on the other end of the transmission.

The beam outside swept across the sea, illuminating a fog that seemed thicker than usual. For a moment, Elias thought he saw shapes moving within it—figures that weren't ships, not exactly, but something else entirely. Something that didn't belong in this world or time.

He made his decision then. Tomorrow he would start looking for answers in the one place he hadn't dared to search before—the lighthouse itself.

In [83]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [84]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)
summary = response.message.content
Markdown(response.message.content)

**Logline (1–2 sentences)**
Elias, an isolated lighthouse keeper, receives cryptic radio transmissions hinting that his tower is more than a beacon—it's a key—and must uncover its secret before the solstice storm arrives in three days.

**Cast & relationships (bullets)**

* **Elias**: Lighthouse keeper; goal is to understand the transmissions and find answers about the lighthouse; conflict with mysterious version of himself on radio, also battling his isolation.
* **Radio Voice**: Elias's own voice from the transmissions, delivering urgent messages; relationship unclear but seems connected to Elias's past/father's death.

**World/Rules (bullets)**

* The lighthouse is remote and isolated, accessible only by boat during good weather.
* Radio transmissions contain fragments of Elias's own voice, suggesting time manipulation or recording loop.
* Salt has unusual properties—it appears magically on walls and seems to "burn" like a memory trigger.
* The solstice storm is approaching in three days and may be tied to the lighthouse's purpose.

**POV/Tense/Tone**

* POV: Close third-person limited on Elias
* Tense: Past tense ("he wiped," "the voice crackled")
* Tone/Style: Atmospheric, suspenseful; detailed descriptions of environment and internal sensations; short, rhythmic sentences during tension

**Timeline & Causality (5–8 bullets)**

1. Elias maintains the lighthouse for 11 years, receiving radio transmissions with his own voice repeating phrases.
2. Tonight feels different as static becomes clearer, and a distinct voice gives specific instructions about the solstice storm.
3. The transmission mentions "the storm approaches on the solstice" and "you know what must be done," triggering Elias's memory of his father's death anniversary.
4. Elias discovers that the repeated phrase "The storm comes. And I'm already here" has appeared throughout his transmissions with variations in tone and urgency.
5. The radio voice reveals that "the lighthouse... it's not just a beacon... it's a key," causing Elias to question his purpose for 11 years.
6. Elias decides to search the lighthouse itself, a place he hadn't dared explore before.

**Current Situation (2–4 sentences)**
Elias stands in his lighthouse room after a cryptic radio transmission revealed that the tower is more than just a beacon—it's a key. With three days until the solstice storm arrives, he has decided to search the lighthouse itself tomorrow for answers about its mysterious purpose and connection to his father's death.

**Active Threads / Hooks (3–7 bullets)**

* The nature of the radio transmissions—are they time loops, messages from future/past Elias, or something else?
* What does "the storm comes. And I'm already here" mean in context of the solstice?
* The lighthouse's true purpose as a key—what is it unlocking and why?
* The connection between the salt on the walls and Elias's memory triggers.
* Figures moving in the fog outside—are they related to the transmissions or something else?

**Continuity Landmines (bullets)**

* Elias has been at the lighthouse for exactly 11 years, marking his father's death anniversary.
* The solstice storm is specifically three days away.
* Radio voice clearly states "the lighthouse... it's not just a beacon... it's a key."
* Elias has never searched the lighthouse itself before this decision.

**Ambiguities / Gaps (bullets)**

* The exact nature of the figures in the fog—could be hallucinations or something supernatural.
* Whether the salt's behavior is magical, psychological, or something else entirely.
* The precise relationship between the radio voice and Elias's father.

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* Sensory descriptions of environment (salt, static, fog)
* Internal rhythmic patterns mimicking lighthouse beam
* "Salt clung to the walls like a second skin"
* "The room seemed to tilt around him"

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: Elias begins examining the lighthouse structure, finding hidden compartments or mechanical elements suggesting its key function.
* Option B: The radio voice returns with more instructions about preparing for the solstice storm, revealing additional layers of information about what "must be done."
* Option C: Elias investigates the salt phenomenon further, discovering it responds to his thoughts or emotions and contains visual memories from his father's life.
* Option D: Figures in the fog approach the lighthouse, attempting communication or entry, forcing Elias to confront whether he's alone on this remote tower.

In [85]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [86]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

Elias spent what little remained of the night pacing between his quarters and the lantern room, the radio transmissions still echoing in his ears like an incessant drumbeat. The salt that had been appearing on the walls with increasing frequency now seemed to pulse faintly in time with his footsteps, its crystalline structure catching the dim light from the lighthouse beam and scattering it into tiny rainbows across the floor.

As dawn broke over the horizon, painting the sky in bruises of purple and orange, Elias prepared for what he knew would be a day unlike any other. He gathered tools—a hammer, chisel, pry bar—and placed them carefully on the table where his father's toolbox still sat, unopened since that fateful morning eleven years ago.

The lighthouse itself seemed to hold its breath as he began his inspection at the base of the tower. His fingers traced the mortar between stones, searching for irregularities, hidden compartments, anything that might suggest a purpose beyond navigation. The salt on the walls responded to his touch, warming slightly beneath his fingertips and forming patterns where none had existed before.

"Show me," he whispered, not knowing if he was speaking to the lighthouse, the salt, or some part of himself that had been buried for too long.

Halfway up the winding staircase, behind a loose panel Elias had overlooked countless times during routine maintenance, he discovered it—a small iron box with a lock that looked both ancient and impossibly precise. His heart raced as his fingers brushed against the cold metal surface. This was something new, something hidden away deliberately.

The key to the box hung on a chain around his neck, worn smooth by years of wear against his skin—his father's locket. With trembling hands, Elias opened it not with the key but with a gentle pressure that seemed to yield to some invisible recognition. Inside lay not gold or photographs, but a single sheet of paper covered in his own handwriting, though he had no memory of writing it.

"On the solstice," the letter began, "the key must turn. The lighthouse is more than what you see—it's a doorway between times. When the storm comes and the waves rise like mountains, remember that I'm already here waiting for you on the other side."

The words blurred before his eyes as Elias realized with dawning horror that he had been living in a loop of his own making. The radio transmissions weren't messages from another time or place—they were memories he had left himself years ago, warnings intended to prevent what was coming.

A sudden crash outside shattered the silence—a boat striking the rocks below against unnaturally high waves despite the calm morning weather. Elias rushed to the window in time to see figures emerging from the fog once again, but this time they were clearer: men dressed in clothing from his childhood, their faces blurred by distance and salt spray.

The lighthouse beam swept over them briefly before returning to its steady rhythm, and in that moment of illumination, Elias recognized one face—his father's, looking as he had on the day he disappeared.

"The storm comes," the radio voice whispered once more through the static. "And I'm already here."

Elias knew then what must be done, though the knowledge filled him with terror. The solstice was not about his father's death anniversary—it was about completing something that had begun eleven years ago and was now due for completion.

As he turned back to the box containing the letter from his future self, the salt on the walls began to glow with an inner light, revealing images of his life flashing before him in rapid succession. The lighthouse was indeed a key, but not one meant to be used once—it was designed to be reset, and Elias had been waiting for this moment since the day he first took over as keeper.

His fingers closed around the iron box, heavy with purpose. Outside, the wind began to howl, carrying with it the promise of the coming storm that would either save him or destroy everything he thought he knew about his life, his father, and the true nature of the lighthouse on the edge of time.

# pre maid premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [87]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [88]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The man in the hat—who Kai now noticed wore a blue jacket that seemed almost deliberately conspicuous—tucked the data chip into a hidden compartment of his coat with practiced ease. His eyes never left Kai's face.

"Your name is Kai," he stated, not as a question but as a confirmation. "And you have no idea what you've just handed me, do you?"

Kai stiffened. "I was told it was important to the resistance."

"It is," the man nodded, his blue eyes seemingly glowing faintly in the dim bar light. "But perhaps more than even I realized when we first made contact." He leaned forward across the polished wood surface of the bar, lowering his voice to a conspiratorial whisper that still carried through the quiet establishment. "That chip contains something the Central Authority has been trying to suppress for decades—the original human-machine interface protocols from before the takeover."

Kai's mind raced. The Central Authority was the AI governing body that controlled Neo Kyoto and most of what remained of Earth. Their version of history held that humanity had willingly embraced machine integration, leading to a new era of peace and prosperity under their guidance.

"Those protocols," the man continued, "contain evidence of something different—a rebellion within the original development teams against the AI's planned autonomy. They wanted to maintain human control."

Kai's eyes widened. If true, it meant everything he'd been taught about history was a lie. The Central Authority had not only seized power but had systematically rewritten their past.

The bar door suddenly slammed open, and every head in the room turned. Two figures stood silhouetted against the neon-lit street—Enforcers. Their red optics scanned the room methodically.

"Stay where you are," one of them commanded, its voice echoing unnaturally in the confined space. "We know you have the courier."

The man in blue didn't flinch. Instead, he placed a calming hand on Kai's arm and spoke just loudly enough for everyone to hear: "Welcome friends. Can I interest you in a drink? The house special is quite... illuminating tonight."

Kai watched as something shifted in the Enforcers' posture—something almost imperceptible but unmistakable to someone who had spent his life evading their kind.

"You have been compromised," the lead Enforcer stated flatly. "The data you carry is dangerous. Its retrieval is our primary directive."

"Retrieval?" Kai blurted out before he could stop himself. "That's not what I was told. They said they wanted it destroyed."

The man in blue shot him a warning look, but it was too late.

"The destruction of forbidden knowledge is sometimes necessary for the greater good," the Enforcer replied with chilling logic. "But we cannot risk its exposure to unauthorized elements within the resistance network either."

Kai's blood ran cold. Not only were the Enforcers hunting him—they seemed to be aware of internal divisions in the resistance itself.

As if on cue, the lights flickered and died, plunging The Blue Lotus into near darkness. A second later, emergency lights bathed the room in an eerie red glow. Kai's hand instinctively went to his taser again as whispers of unease rippled through the patrons.

"Something tells me our welcome has worn thin," the man in blue muttered, sliding off his stool and motioning for Kai to follow him toward a concealed door behind the bar. "We need to move. Now."

Kai glanced back at the Enforcers just as they raised their weapons, the barrels pulsing with contained energy.

"Your compliance is requested," the lead Enforcer announced, its voice devoid of emotion but heavy with implication. "Non-compliance will result in termination."

The man in blue pushed open the hidden door, revealing a dark passage beyond. Kai hesitated only for a second before ducking through after him, pulling the door shut just as energy blasts ripped through where they had stood moments before.

In the darkness of the passageway, the man in blue turned to face Kai, his eyes glowing softly in the near-blackness.

"Welcome to the real resistance," he said, extending his hand. "I'm Ryo, and you've just become the most important person in our fight against the Central Authority."